<a href="https://colab.research.google.com/github/jasmine400/nlp-/blob/main/association_rule_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My\ Drive/associate
!ls

/content/gdrive/My Drive/associate
'association rule .ipynb'   book_tags.csv   tags.csv
 books.csv		    ratings.csv     to_read.csv


In [ ]:
booksRaw =  pd.read_csv("books.csv")
booksRaw.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [ ]:
import numpy as np
import pandas as pd

#Import datasets
booksRaw =  pd.read_csv("books.csv")
tagsRaw = pd.read_csv("tags.csv")
bookTagsRaw = pd.read_csv("book_tags.csv")
ratingsRaw = pd.read_csv("ratings.csv")
to_readRaw = pd.read_csv("to_read.csv")

#Merge bookTags and tags
all_tags = pd.merge(bookTagsRaw, tagsRaw, left_on='tag_id', right_on='tag_id', how='inner')
all_tags_grouped = all_tags.groupby("goodreads_book_id")
all_tags.tail(100)

#Group ratings by user
ratingsGrouped = ratingsRaw.sort_values('user_id').groupby(by='user_id')
ratingsGrouped.get_group(1)

,book_id,user_id,rating
117889,1180,1,4
488112,4893,1,3
625717,6285,1,4


In [ ]:
class UserProfile:
  def __init__(self, user_id):
    self.userID = user_id;
    self.allTags = [];
    self.highWords = set()
    self.lowWords = set()

  def makeUserProfile(self):
      #Separate high rated books and low rated books
      ratedBooks = ratingsGrouped.get_group(self.userID)
      high = ratedBooks[ratedBooks.rating >= 3]
      low = ratedBooks[ratedBooks.rating < 3]

      for index, row in high.iterrows():
        #Add tags to list of keywords
        taglist = all_tags[all_tags.goodreads_book_id == row['book_id']].tag_name.tolist()

        #Add author names to list of keywords
        authors = booksRaw[booksRaw.book_id == row['book_id']].authors.tolist()
        if (authors != []):
          authors = authors[0].split(",")
          taglist.extend(authors)

        self.highWords.union(set(taglist))

        #Add rating keyword to transaction
        taglist.append("like")

        #Add keyword list to list of all transactions
        if (taglist != []):
          self.allTags.append(taglist)

      #repeat this process for the low rated books
      for index, row in low.iterrows():
        taglist = all_tags[all_tags.goodreads_book_id == row['book_id']].tag_name.tolist()

        authors = booksRaw[booksRaw.book_id == row['book_id']].authors.tolist()
        if (authors != []):
          authors = authors[0].split(",")
          taglist.extend(authors)

        self.lowWords.union(set(taglist))

        taglist.append("dislike")
        if (taglist != []):
          self.allTags.append(taglist)

      #Remove intersection of low and high keywords from all transactions
      intersection = self.highWords.intersection(self.lowWords)
      for i in self.allTags:
        j = 0
        while j < len(i):
          if i[j] in intersection:
            i.pop(j)
          j += 1

user = UserProfile(17566)
user.makeUserProfile()
print(user.allTags)

[['like'], ['like'], ['like'], ['like'], ['like'], ['to-read', 'favorites', 'currently-reading', 'young-adult', 'fiction', 'books-i-own', 'owned', 'favourites', 'owned-books', 'all-time-favorites', 'default', 'my-books', 'i-own', 'audiobook', 'favorite-books', 'novels', 'favorite', 'audiobooks', 'my-library', 'english', 'british', 'read-in-2016', 'own-it', 'library', 'audio', 'novel', 'read-in-2015', 'shelfari-favorites', 'kindle', 'romance', 'to-buy', 'read-in-2014', 'ebook', 'contemporary', 'england', 'audio-books', 'read-in-2013', 'read-in-2011', 'borrowed', 'female-author', 'contemporary-fiction', 'wish-list', 'recommended', 'adult', 'book-club', 'adult-fiction', 'literature', 'abandoned', 'audible', 'ebooks', 'audio-book', 'did-not-finish', 'to-read-fiction', 'history', 'historical', 'didn-t-finish', 'australian', 'listened-to', 'bookclub', 'literary', 'read-in-2012', 'europe', 'britain', 'uk', 'want-to-read', 'survival', 'school', 'general-fiction', 'historical-fiction', 'literar

In [ ]:

from effiecient_apriori import apriori
print("Length of allTags: ", sum(len(tag) for tag in user.allTags))
allItemsets, allRules = list(apriori(user.allTags, min_confidence=0.3,min_support=0.13))
print("Without minimum confidence and support, there would be over rules! Here we narrow it down to just", len(allRules),".")
print(allRules)